In [1]:
!pip install hanja

     |████████████████████████████████| 120 kB 25.1 MB/s 
     |████████████████████████████████| 265 kB 29.1 MB/s 
     |████████████████████████████████| 242 kB 43.8 MB/s 
     |████████████████████████████████| 280 kB 42.9 MB/s 
     |████████████████████████████████| 242 kB 32.3 MB/s 
     |████████████████████████████████| 242 kB 24.5 MB/s 
     |████████████████████████████████| 229 kB 30.3 MB/s 
     |████████████████████████████████| 229 kB 45.5 MB/s 
INFO: pip is looking at multiple versions of pytest-cov to determine which version is compatible with other requirements. This could take a while.
  Created wheel for hanja: filename=hanja-0.13.3-py3-none-any.whl size=128423 sha256=960d4742e771e204931cfdec38d37172d170dc282e1f7f6ddaf8bd4187109701
  Stored in directory: /root/.cache/pip/wheels/70/08/88/f9cd32ddb92f5c3061cf16f068c842dc558d2f66a9c943b51a
  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp37-cp37m-linux_x86_64.whl size=44117 sha256=2ff9afedb78ffb7242e7df906feb9677a00b

In [2]:
import unicodedata
import hanja
import argparse
from hanja import hangul
import pandas as pd
import csv
import json
import re
from  pprint import pprint
from google.colab import drive
drive.mount('/content/drive')
from glob import glob
import collections

Mounted at /content/drive


In [3]:
sample = sorted(glob("/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/*.txt"))
# pprint(sample)

In [25]:
# 파일 불러오기

fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 1차/284여성(예산결산기금심사)소위01.txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 1차/311법사(법안심사제1)소위01(12.11.15).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 1차/311환노(법안심사)소위01(12.9.17).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 1차/320환노(예산결산기금심사)소위03(13.11.28).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 1차/321교문(예산결산기금심사)소위03(13.12.16).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 1차/321국토(교통법안심사)소위01(13.12.12).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 1차/322국토(민투mrg대책)소위01(14.2.26).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 1차/322국토(철도산업발전)소위01(14.2.7).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 1차/323국토(철도산업발전)소위02(14.4.10).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 1차/320교문(청원심사)소위01(13.11.18).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 1차/346외통(법안심사)소위02(16.10.28.).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 1차/347안행(안전및선거법심사)소위01(16.12.20.).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 1차/354환노(고용노동)소위03(17.9.28.).txt"

# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/311교과(법안심사)소위03(12.9.24).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/311국토(예산결산기금심사)소위02(12.11.7).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/311법사(예산결산심사)소위03(12.11.21).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/311법사(예산결산심사)소위04(12.12.3).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/311외통(법안심사)소위03(12.11.19).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/311재정(경제재정)소위03(12.9.24).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/311재정(예산결산기금심사)소위03(12.11.6).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/311재정(예산결산기금심사)소위08(12.11.15).txt"

# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/311재정(예산결산기금심사)소위09(12.11.20).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/311행안(법안심사)소위02(12.9.19).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/313문방(법안심사)소위01(13.2.15).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/315국방(예산결산심사)소위01(13.4.23).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/315여성(유엔협약이행정책심사)소위01(13.4.23).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/315국토(교통법안심사)소위01(13.4.23).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/315재정(예산결산기금심사)소위03(13.5.3).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/315재정(조세)소위02(13.4.18).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/315환노(법안심사)소위02(13.4.17)(공청회).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/320국토(청원심사)소위01(13.10.8).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/321법사(예산결산심사)소위02(13.12.12).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/324법사(법안심사제2)소위01(14.4.22).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/329법사(법안심사제1)소위01(14.11.6).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/329재정(예산결산기금심사)소위03(14.11.11).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/329환노(법안심사)소위06(14.12.8).txt"

# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/334안행(예산결산기금심사)소위02(15.7.3).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/340국방(법률안심사)소위01(16.2.16).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/346기재(조세)소위08(16.11.28.).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/348법사(법안심사제1)소위01(17.1.18.).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/349국방(법률안심사)소위02(17.2.13.)(공청회).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/337국방(청원심사)소위01(15.11.18).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/340국방(법률안심사)소위01(16.2.16).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/340국방(법률안심사)소위01(16.2.16).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/353기재(조세)소위01(17.8.22.).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/354여성(법안심사)소위01(17.9.18.)(공청회).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/356여성(국제규약점검)소위01(18.2.13.).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/371환노(예산결산기금심사)소위05(19.11.7.).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/377행안(예산.결산및기금심사)소위01(20.4.28.).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/364국토(국토법안심사)소위04(18.12.07.)(공청회).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/364국방(병역특례제도개선)소위02(18.11.29.).txt"
# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 2차/349국방(법률안심사)소위02(17.2.13.)(공청회).txt"



# fname = "/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 1,2차/332법사(법안심사제1)소위02(15.5.1).txt"


with open(fname, 'r', encoding='utf-8-sig') as file:
  txt = file.readlines()
  pass
# pprint(txt)

In [26]:
def clean_up(txt):
  tt = []
  ttt = []
  text = []
  for page in txt:
    sent = page.strip()    
    sent = sent.split("\n")
    tt += sent
  for i in range(len(tt)):
    if tt[i].endswith("."):            
      index = tt[i].rfind('.')
      ttt.append(tt[i][:index] + '. ')
    elif tt[i].endswith("?"):            
      index = tt[i].rfind('?')
      ttt.append(tt[i][:index] + '? ')
    elif tt[i].endswith("!"):            
      index = tt[i].rfind('!')
      ttt.append(tt[i][:index] + '! ')
    else:
      ttt.append(tt[i])
# 이름의 성(김,임,춘..등) 같은경우 외장함수 hanja를 이용해도 한자 코드가 일정한 음으로 변경되지 않을 수 있기 때문에
# 미리 변환해 두어 오류를 방지 ex) 金 -> 김, 금 : 김으로 통일
  for han in range(len(ttt)):    
    ktext = ttt[han].replace('臨', '임').replace("委","위").replace("員",'원').replace("金","김").replace("金","김").replace('淇','기').replace('春','춘').replace("柳","유").replace("李","이").replace("梁","양").replace("羅","나").replace("利","이").replace("勞","노").replace("盧","노").replace("樂","락").replace("樂","락").replace("龍","용").replace("沈","심").replace("呂","려").replace("寧","영")      
    text.append(ktext)    
  return text
text = clean_up(txt)
# pprint(text)

In [27]:
def clean_up_or(txt):
  # 같은 한자여도 유니코드가 다를 수 있다. 때문에 하나의 형태의 유니코드로 통일시켜 아래와 같이 적용***
  # 다른 한자로 읽어 들이는 오류를 미리 방지. ex) 金 -> 김, 금 : 한자가 같아도 유니코드가 다르다.
  a = unicodedata.normalize('NFC','金')
  b = unicodedata.normalize('NFC',"樂")
  c = unicodedata.normalize('NFC',"柳")
  d = unicodedata.normalize('NFC',"劉")


  text_or =[]  
  tt = []
  ttt = []
  for page in txt:
    sent = page.strip()
    sent = sent.split("\n")      
    tt += sent
  for i in range(len(tt)):
    if tt[i].endswith("."):            
      index = tt[i].rfind('.')
      ttt.append(tt[i][:index] + '. ')
    elif tt[i].endswith("?"):            
      index = tt[i].rfind('?')
      ttt.append(tt[i][:index] + '? ')
    elif tt[i].endswith("!"):            
      index = tt[i].rfind('!')
      ttt.append(tt[i][:index] + '! ')
    else:
      ttt.append(tt[i])
  for han in range(len(ttt)): 
    # ***
    ktext = ttt[han].replace("金",a).replace("金",a).replace("樂",b).replace("樂",b).replace("柳",c).replace("柳",c).replace("劉",d).replace("劉",d)
    text_or.append(ktext) 
  return text_or
text_or = clean_up_or(txt)
# pprint(text_or)

In [28]:
# 본격적 대화(한글화버전)
def dialog_extractor(text):
    dialog = []
    doc = []
    ### 대사 추출 프로그램    
    for page in text:  
        sents = page.split('\n')              
        doc.append(sents)    
    s_start = False
    s_end = False
    break_early =False  
    for sid in range(len(text)):                    
        sent = text[sid].lstrip()        
        if sent.startswith('◯'):
            s_start = sid
            break_early = True
            break                          
        if break_early:
            break
    for sid in range(len(text)):
        sent = text[sid].lstrip()
        if sent.endswith('산회)') or sent.endswith('중지)'):
            s_end = sid       
    for sid in range(len(text)):
        sent = text[sid].lstrip()        
    break_early = False
    doc = []    
    for sid in range(len(text)):
      if sid >= s_start and sid <= s_end - 1:
        sent = text[sid].lstrip()        
        sent = hanja.translate(sent,'substitution').replace("金","김").replace("李","이").replace("梁","양").replace("羅","나").replace("利","이").replace("勞","노").replace("樂","락").replace("盧","노").replace("樂","락").replace("龍","용").replace("沈","심").replace("呂","려").replace("寧","영").replace("宅","택").replace("劉","유")
        if sent.startswith("1.") or sent.startswith("2.") or sent.startswith("3.") or sent.startswith("4.") or sent.startswith("5.") or sent.startswith("가.") or sent.startswith("나.") or sent.startswith("다.") or sent.startswith("라."):
          continue          
        if sent.endswith(")"):      
          continue          
        if sent.startswith('◯'):
          speaker = ' '.join(text[sid].split(' ')[0:3])
          tokens = ' '.join(text[sid].split(' ')[3:])
          sent1 = sent.replace('◯','◯ ')          
          doc.append(sent1)      
        else:
          doc.append(sent)
    doc_p = ''.join(doc)    
    doc_ps = doc_p.split('◯')
    rev_doc = []
    for s in doc_ps:              
            if s == '':
                pass
            elif s.startswith(" "):
                s = '◯'+s                
                rev_doc.append(s)                
            else:                
                rev_doc.append(s)
    dialog += rev_doc
    
    return dialog
dialog = dialog_extractor(text)
# pprint(dialog)

In [29]:
# 본격적 대화(원본)
def dialog_extractor_or(text_or):
    dialog_or = []
    doc = []
    ### 대사 추출 프로그램       
    for page in text_or:  
        sents = page.split('\n')              
        doc.append(sents)    
    s_start = False
    s_end = False
    break_early =False  
    for sid in range(len(text_or)):                    
        sent = text_or[sid].lstrip()        
        if sent.startswith('◯'):
            s_start = sid
            break_early = True
            break                          
        if break_early:
            break
    for sid in range(len(text_or)):
        sent = text_or[sid].lstrip()
        if sent.endswith('산회)') or sent.endswith('중지)'):
            s_end = sid

    for sid in range(len(text_or)):
        sent = text_or[sid].lstrip()        
    break_early = False
    doc = []
    for sid in range(len(text_or)):
      if sid >= s_start and sid <= s_end - 1:
        sent = text_or[sid].lstrip()
        if sent.startswith("1.") or sent.startswith("2.") or sent.startswith("3.") or sent.startswith("4.") or sent.startswith("5.") or sent.startswith("가.") or sent.startswith("나.") or sent.startswith("다.") or sent.startswith("라."):
          continue          
        if sent.endswith(")"):      
          continue          
        if sent.startswith('◯'):
          speaker = ' '.join(text_or[sid].split(' ')[0:3])
          tokens = ' '.join(text_or[sid].split(' ')[3:])
          sent1 = sent.replace('◯','◯ ')          
          doc.append(sent1)
        else:
          doc.append(sent)
    doc_p = ''.join(doc)    
    doc_ps = doc_p.split('◯')
    rev_doc = []
    for s in doc_ps:               
            if s == '':
                pass
            elif s.startswith(" "):
                s = '◯'+s                
                rev_doc.append(s)                
            else:                
                rev_doc.append(s)
    dialog_or += rev_doc
    
    return dialog_or
dialog_or = dialog_extractor_or(text_or)
# pprint(dialog_or)

In [30]:
# 대화에 참여자 이름, 직위(한글화)
def speaker_extractor(dialog):
  global persons
  persons =[]
  occups = []
  speaker_ex = []
  for i in range(len(dialog)):
    dialog_s = dialog[i].split(' ')    
    first_t = dialog_s[1]
    second_t = dialog_s[2]
    # print(dialog_s)
    # if first_t.endswith("관") or first_t.endswith("장") or first_t.endswith("위원") or first_t.endswith("참고인") and len(first_t) >= 3:
    if second_t != "의원" and second_t != "위원" and second_t != "委員" and second_t != "議員":
      occup = first_t.replace("◯ ","")
    else:
      person = first_t.replace("◯ ","")
    if second_t == "의원" or second_t == "위원" or second_t == "委員" or second_t == "議員":
      occup = second_t
    else:
      person = second_t    
    
    persons.append(person)
    occups.append(occup)   
  po = []
  for j in range(len(persons)):
    sp = persons[j]+ " " + occups[j]
    po.append(sp)
  
    
  n_po = []
  for v in po:
    if v not in n_po:
      n_po.append(v)    
  for w in range(len(n_po)):
    n_pos = n_po[w].split(' ') 
    speaker_ex.append(n_pos)        
  return speaker_ex
speaker_ex = speaker_extractor(dialog)
# pprint(speaker_ex)

In [31]:
# 대화에 참여자 이름, 직위
def speaker_extractor_or(dialog_or): 
  
  global personss
  personss =[]
  occups = []
  speaker_ex_or = []
  for i in range(len(dialog_or)):
    dialog_s = dialog_or[i].split(' ')    
    first_t = dialog_s[1]
    second_t = dialog_s[2]    
    # if first_t.endswith("관") or first_t.endswith("장") or first_t.endswith("위원") or first_t.endswith("참고인") and len(first_t) >= 3:
    if second_t != "의원" and second_t != "위원" and second_t != "委員" and second_t != "議員":
      occup = first_t.replace("◯ ","")
    else:
      person = first_t.replace("◯ ","")
    if second_t == "의원" or second_t == "위원" or second_t == "委員" or second_t == "議員":
      occup = second_t
    else:
      person = second_t        
    personss.append(person)
    occups.append(occup)  
  po = []
  for j in range(len(personss)):
    sp = personss[j]+ " " + occups[j]
    po.append(sp)
  # pprint(po)
  n_po = []

  for v in po:
    if v not in n_po:
      n_po.append(v)      
  for w in range(len(n_po)):    
    n_pos = n_po[w].split(' ')    
    speaker_ex_or.append(n_pos) 
  return speaker_ex_or
speaker_ex_or = speaker_extractor_or(dialog_or)
# pprint(speaker_ex_or)

In [32]:
# 발언자, 발언(한글수정)
def utterance_extractor(dialog): 
  utterance_id = []
  utterance_form = []  
  
  for i in range(len(dialog)):
    dialog_s = dialog[i].split(' ')
    # first_s = dialog_s[1:3]
    second_s = dialog_s[4:]
    # spk_id = ' '.join(first_s)
    utter = ' '.join(second_s)
    # utterance_id.append(spk_id)
    utterance_form.append(utter)  
  return persons, utterance_form
utterance_ex = utterance_extractor(dialog)
# pprint(utterance_ex)

In [33]:
# 발언자, 발언(원본)
def utterance_extractor_or(dialog_or): 
  utterance_id = []
  utterance_form = []
  
  for i in range(len(dialog_or)):
    dialog_s = dialog_or[i].split(' ')
    # first_s = dialog_s[1:3]
    second_s = dialog_s[4:]
    # spk_id = ' '.join(first_s)
    utter = ' '.join(second_s)
    # utterance_id.append(spk_id)
    utterance_form.append(utter)
  return personss, utterance_form
utterance_ex_or = utterance_extractor_or(dialog_or)
# pprint(utterance_ex_or)

In [34]:
# utterance info
def dialog_formatting(utterance_ex, utterance_ex_or):  
  dialog_json = []    
  utterance_id = utterance_ex_or[0]
  utterance_form = utterance_ex[1]
  utterance_form_or = utterance_ex_or[1]  
  # pprint(utterance_form_or)
  for i in range(len(utterance_ex_or[0])):
    d = {}
    d['speaker'] = utterance_id[i]
    d['utterance'] = utterance_form[i] 
    d['utterance_or'] = utterance_form_or[i]      
    
    dialog_json.append(d)      
  # result.append(d)
  return dialog_json
  
dialog_json = dialog_formatting(utterance_ex, utterance_ex_or)
# pprint(dialog_json)

In [35]:
# speaker info
def speaker_list_extractor(speaker_ex,speaker_ex_or):
  global p_name
  p_name = []  
  speaker_list = []  
    
  for i in range(len(speaker_ex)):        
    name = speaker_ex[i][0]        
    position = speaker_ex[i][1] 
    # o_name = speaker_ex_or[i][0]
    if position != "소위원장대리":
      d = {}      
      d['id'] = name
      # d['age'] = "NA"
      d['occupation'] = position
      # d['sex'] = "NA"
      # d['birthplace'] = "NA"
      # d['principal_residence'] = "NA"
      # d['current_residence'] = "NA"
      # d['original_speaker_id'] = o_name 
      speaker_list.append(d) 
      p_name.append(d['id']) 
    
  return speaker_list
speaker_list = speaker_list_extractor(speaker_ex,speaker_ex_or)
# pprint(speaker_list)

In [38]:
# speaker info(original)
def speaker_list_extractor_or(speaker_ex,speaker_ex_or):
  global p2_name
  p2_name = []  
  speaker_list = []         
  for i in range(len(speaker_ex)):        
    name = speaker_ex[i][0]        
    position = speaker_ex[i][1] 
    o_name = speaker_ex_or[i][0]
    if position != "소위원장대리":
      d = {}      
      d['id'] = name
      # d['age'] = "NA"
      d['occupation'] = position
      # d['sex'] = "NA"
      # d['birthplace'] = "NA"
      # d['principal_residence'] = "NA"
      # d['current_residence'] = "NA"
      d['original_speaker_id'] = o_name 
      speaker_list.append(d)
      p2_name.append(d['id'])   
  return speaker_list
speaker_list__ = speaker_list_extractor_or(speaker_ex,speaker_ex_or)
# pprint(speaker_list__)

[{'id': '김상희', 'occupation': '소위원장', 'original_speaker_id': '김상희'},
 {'id': '김귀순', 'occupation': '수석전문위원', 'original_speaker_id': '김귀순'},
 {'id': '김옥이', 'occupation': '위원', 'original_speaker_id': '金玉伊'},
 {'id': '김태석', 'occupation': '여성부기획조정실장', 'original_speaker_id': '김태석'},
 {'id': '이정선', 'occupation': '위원', 'original_speaker_id': '이정선'},
 {'id': '곽정숙', 'occupation': '위원', 'original_speaker_id': '곽정숙'},
 {'id': '손숙미', 'occupation': '위원', 'original_speaker_id': '손숙미'},
 {'id': '이복실', 'occupation': '여성부권익증진국장', 'original_speaker_id': '이복실'},
 {'id': '정봉협', 'occupation': '여성부여성정책국장', 'original_speaker_id': '정봉협'},
 {'id': '김춘진', 'occupation': '위원', 'original_speaker_id': '김춘진'},
 {'id': '이성선', 'occupation': '여성부인력개발사업과장', 'original_speaker_id': '이성선'},
 {'id': '김권영', 'occupation': '여성부복지지원과장', 'original_speaker_id': '김권영'}]


In [50]:
def speaker_extractor2_or(text,speaker_list, speaker_list__, speaker_ex):
  s_start = False  
  ss_start = False  
  break_early = False
  s_sentence = []  
  ss_sentence = []  
  sss_sentence =[]
  c_sentence = []
  d_sentence = []
  speaker_list2 = []  
  speaker_list2__2 = []    
  e_sentence = []
  f_sentence = []
  g_sentence = []
  h_sentence = []
  B_member = []
  part2 = False
  part3 = False
  part4 = False
  part5 = False  
  c_end = False
  d_end = False
  e_end = False
  f_end = False
  g_end = False
  h_end = False
  role = ""
  b = 0
  
  # 산회) 이하 제~일) 이상 문장 뽑기
  # for i in range(len(text)):
  #   if text[i].endswith("산회)") or text[i].endswith('중지)'):
  #     s_start = i + 1  
  for i in range(len(text)):
    if text[i].startswith("◯출석") or text[i].startswith("○출석"):
      s_start = i - 1       
      break_early = True
      break 
  break_early = False  
  for i in range(len(text)):
    if text[i].startswith("제") and (text[i].endswith('일)') or text[i].endswith('차')):
      if text[i - 1] == '' and text[i-2] == '' and text[i-3] =='':
        s_end = i - 3
        break_early = True
        break
      break_early = False
      if text[i - 1] == '' and text[i-2] == '':
        s_end = i - 2
        break_early = True
        break
      break_early = False
      if text[i - 1] != '':
        s_end = i
        break_early = True
        break
      break_early = False  
  for sid in range(len(text)):    
    if sid >=s_start and sid < s_end:      
      sub_sentences = hanja.translate(text[sid],"substitution").replace("金","김").replace("李","이").replace("梁","양").replace("羅","나").replace("利","이").replace("勞","노").replace("樂","락").replace("盧","노").replace("樂","락").replace("龍","용").replace("沈","심").replace("宅","택")
      s_sentence.append(sub_sentences) 

  # 출석 위원
  for j in range(len(s_sentence)):    
    if s_sentence[j].startswith("◯") or s_sentence[j].startswith("○"):
      m =  j + 1
      A_member = s_sentence[m].split("  ")      
      break_early = True
      break
  break_early = False   
  # p_name2 = []  
  for t in range(len(A_member)):    
    if A_member[t] not in p_name:      
      name = A_member[t]
      d = {}      
      d['id'] = name
      # d['age'] = "NA"
      d['occupation'] = "위원"
      # d['sex'] = "NA"
      # d['birthplace'] = "NA"
      # d['principal_residence'] = "NA"
      # d['current_residence'] = "NA"
      d['original_speaker_id'] = name 
      speaker_list2.append(d)  
  
  for u in range(len(s_sentence)):
    if s_sentence[u] == "" and s_sentence[u+1] =="":
      first_line_end = u      
      break_early = True
      break 
  break_early = False 
    
  # 위원 아닌 출석 의원(X인)
  for e in range(len(s_sentence)):
    if e > m and e < first_line_end:
      ss_sentence += s_sentence[e].split("\n")      
  for a in range(len(ss_sentence)):    
    # 수정했는데 맞겠지?
    if (ss_sentence[a].startswith("◯") or ss_sentence[a].startswith("○")) and ss_sentence[a].endswith("인)"):      
      if ss_sentence[a].startswith("◯위원") or ss_sentence[a].startswith("○위원"):                
        b = a + 1        
        B_member = ss_sentence[b].split('  ')
        occup = "의원"
        break       
      else:
        b = a + 1
        B_member = ss_sentence[b].split('  ') 
        occup = "위원"
        break     
  for y in range(len(B_member)): 
    if B_member[y] not in p_name:
      name = B_member[y]
      d = {}      
      d['id'] = name
      # d['age'] = "NA"
      d['occupation'] = occup
      # d['sex'] = "NA"
      # d['birthplace'] = "NA"
      # d['principal_residence'] = "NA"
      # d['current_residence'] = "NA"
      d['original_speaker_id'] = name 
      speaker_list2.append(d)
  # 위원 아닌 출석 의원(X인)2
  if ss_sentence[b+1].endswith("인)") == True:
    for e in range(len(s_sentence)):
      if e > m and e < first_line_end:
        ss_sentence += s_sentence[e].split("\n")      
    for a in range(len(ss_sentence)):    
      # 수정했는데 맞겠지?
      if (ss_sentence[a].startswith("◯") or ss_sentence[a].startswith("○")) and ss_sentence[a].endswith("인)"):      
        if ss_sentence[a].startswith("◯위원") or ss_sentence[a].startswith("○위원"):                
          b = a + 1        
          B_member = ss_sentence[b].split('  ')
          occup = "의원"
        else:
          b = a + 1
          B_member = ss_sentence[b].split('  ') 
          occup = "위원"
          
    for y in range(len(B_member)): 
      if B_member[y] not in p_name:
        name = B_member[y]
        d = {}      
        d['id'] = name
        # d['age'] = "NA"
        d['occupation'] = occup
        # d['sex'] = "NA"
        # d['birthplace'] = "NA"
        # d['principal_residence'] = "NA"
        # d['current_residence'] = "NA"
        d['original_speaker_id'] = name 
        speaker_list2.append(d)
  
  # 출석 전문위원
  for a in range(len(ss_sentence)):    
    if not ss_sentence[a].endswith("인)") and (ss_sentence[a].startswith("◯") or ss_sentence[a].startswith("○")):
      s_start = a+1  
  for z in range(len(ss_sentence)):
    if z >= s_start:
      sss = ss_sentence[z].split("\n")
      sss_sentence += sss    
  count = int(len(sss_sentence) / 2)  
  id_s = range(count,len(sss_sentence))  
  ocu_s = range(0,count)
  for n, m in zip(id_s, ocu_s) :                
    if sss_sentence[n] not in p_name:        
      d = {}
      d['id'] = sss_sentence[n]
      # d['age'] = "NA"
      d['occupation'] = sss_sentence[m]
      # d['sex'] = "NA"
      # d['birthplace'] = "NA"
      # d['principal_residence'] = "NA"
      # d['current_residence'] = "NA"
      d['original_speaker_id'] = sss_sentence[n] 
      speaker_list2.append(d) 
  
  try:  
    ## 위원 의원 아래 참석자      
    for p in range(len(s_sentence)):        
      if s_sentence[p].startswith("◯") and s_sentence[p].endswith("참석자"):      
        part = p + 1
        c = p + 2
        c_part = s_sentence[part]
        break  
    for l in range(len(s_sentence)):    
      if l >= part:      
        if s_sentence[l] == "" and s_sentence[l+1] =="":
          c_end = l
          break
        else:
          c_end = l + 1
    for m in range(len(s_sentence)):
      if m >= part and m < c_end:
        ssss = s_sentence[m].split('\n')      
        c_sentence += ssss
    count = int((len(c_sentence) -1) / 2) + 1  
    id_c = range(count, len(c_sentence))
    ocu_c = range(1,count)
    for n, m in zip(id_c, ocu_c) :                
      if c_sentence[n] not in p_name:        
        d = {}
        d['id'] = c_sentence[n]
        # d['age'] = "NA"
        d['occupation'] = c_part + c_sentence[m]
        # d['sex'] = "NA"
        # d['birthplace'] = "NA"
        # d['principal_residence'] = "NA"
        # d['current_residence'] = "NA"
        d['original_speaker_id'] = c_sentence[n]
        speaker_list2.append(d)

  # 그다음꺼 있으면    
    if c_end != False:
      for h in range(len(s_sentence)):
        if h > c_end:
          if s_sentence[c_end] =="" and s_sentence[c_end+1] =="" and not s_sentence[c_end+2].startswith("◯"):          
            part2 = c_end +2
            d_part = s_sentence[part2]
            break
      if part2 != False:
        for r in range(len(s_sentence)):
          if r >= part2:
            if s_sentence[r] == "" and s_sentence[r+1] =="":
              d_end = r        
              break
            else:
              d_end = r + 1
      if part2 != False:
        for m in range(len(s_sentence)):
          if m >= part2 and m < d_end:
            sss = s_sentence[m].split('\n')      
            d_sentence += sss
        count = int((len(d_sentence) -1) / 2) + 1  
        id_d = range(count, len(d_sentence))
        ocu_d = range(1,count)
        for n, m in zip(id_d, ocu_d) :                
          if d_sentence[n] not in p_name:        
            d = {}
            d['id'] = d_sentence[n]
            # d['age'] = "NA"
            d['occupation'] = d_part + d_sentence[m]
            # d['sex'] = "NA"
            # d['birthplace'] = "NA"
            # d['principal_residence'] = "NA"
            # d['current_residence'] = "NA"
            d['original_speaker_id'] = d_sentence[n]
            speaker_list2.append(d)     
  # 그다음꺼 있으면    2  
    if d_end != False:
      for h in range(len(s_sentence)):
        if h > d_end:
          if s_sentence[d_end] =="" and s_sentence[d_end+1] =="" and not s_sentence[d_end+2].startswith("◯"):          
            part3 = d_end +2
            e_part = s_sentence[part3]        
            break
      # print(part2)
      if part3 != False:
        for r in range(len(s_sentence)):
          if r >= part3:
            if s_sentence[r] == "" and s_sentence[r+1] =="":
              e_end = r        
              break
            else:
              e_end = r + 1
      if part3 != False:
        for m in range(len(s_sentence)):
          if m >= part3 and m < e_end:
            ssssss = s_sentence[m].split('\n')      
            e_sentence += ssssss
        # pprint(d_sentence)
        count = int((len(e_sentence) -1) / 2) + 1  
        id_e = range(count, len(e_sentence))
        ocu_e = range(1,count)
        for n, m in zip(id_e, ocu_e) :                
          if e_sentence[n] not in p_name:        
            d = {}
            d['id'] = e_sentence[n]
            # d['age'] = "NA"
            d['occupation'] = e_part + e_sentence[m]
            # d['sex'] = "NA"
            # d['birthplace'] = "NA"
            # d['principal_residence'] = "NA"
            # d['current_residence'] = "NA"
            d['original_speaker_id'] = e_sentence[n]
            speaker_list2.append(d)
  # 그다음꺼 있으면   3
    if e_end != False:
      for h in range(len(s_sentence)):
        if h > e_end:
          if s_sentence[e_end] =="" and s_sentence[e_end+1] =="" and not s_sentence[e_end+2].startswith("◯"):          
            part4 = e_end +2
            f_part = s_sentence[part4]        
            break
      # print(part2)
      if part4 != False:
        for r in range(len(s_sentence)):
          if r >= part4:
            if s_sentence[r] == "" and s_sentence[r+1] =="":
              f_end = r        
              break
            else:
              f_end = r + 1
      if part4 != False:
        for m in range(len(s_sentence)):
          if m >= part4 and m < f_end:
            sssssss = s_sentence[m].split('\n')      
            f_sentence += sssssss
        # pprint(d_sentence)
        count = int((len(f_sentence) -1) / 2) + 1  
        id_f = range(count, len(f_sentence))
        ocu_f = range(1,count)
        for n, m in zip(id_f, ocu_f) :                
          if f_sentence[n] not in p_name:        
            d = {}
            d['id'] = f_sentence[n]
            # d['age'] = "NA"
            d['occupation'] = f_part + f_sentence[m]
            # d['sex'] = "NA"
            # d['birthplace'] = "NA"
            # d['principal_residence'] = "NA"
            # d['current_residence'] = "NA"
            d['original_speaker_id'] = f_sentence[n]
            speaker_list2.append(d)
  # 그다음꺼 있으면 4
    if f_end != False:
      for h in range(len(s_sentence)):
        if h > f_end:
          if s_sentence[f_end] =="" and s_sentence[f_end+1] =="" and not s_sentence[f_end+2].startswith("◯"):          
            part5 = f_end +2
            g_part = s_sentence[part5]        
            break      
      if part5 != False:
        for r in range(len(s_sentence)):
          if r >= part5:
            if s_sentence[r] == "" and s_sentence[r+1] =="":
              g_end = r        
              break
            else:
              g_end = r + 1
      if part5 != False:
        for m in range(len(s_sentence)):
          if m >= part5 and m < g_end:
            ssssssss = s_sentence[m].split('\n')      
            g_sentence += ssssssss
        # pprint(d_sentence)
        count = int((len(f_sentence) -1) / 2) + 1  
        id_g = range(count, len(f_sentence))
        ocu_g = range(1,count)
        for n, m in zip(id_g, ocu_g) :                
          if g_sentence[n] not in p_name:        
            d = {}
            d['id'] = g_sentence[n]
            # d['age'] = "NA"
            d['occupation'] = g_part + g_sentence[m]
            # d['sex'] = "NA"
            # d['birthplace'] = "NA"
            # d['principal_residence'] = "NA"
            # d['current_residence'] = "NA"
            d['original_speaker_id'] = f_sentence[n]
            speaker_list2.append(d)              
      # print("except")
    # 진술인 참고인 
    for f in range(len(s_sentence)):
      if s_sentence[f].startswith("◯") and s_sentence[f].endswith("참고인"):
        role = s_sentence[f].split(" ")[1]            
        ss_start = f + 1         
    if ss_start != False:
      # print(s_start)
      for j in range(len(s_sentence)):
        if j >= ss_start:
          ee = s_sentence[j].split("(")
          eee = ee[0].split('\n')
          e_sentence += eee      
      for n in range(len(e_sentence)) :                
        if e_sentence[n] not in p_name:        
          d = {}
          d['id'] = e_sentence[n]
          # d['age'] = "NA"
          d['occupation'] = role  
          # d['sex'] = "NA"
          # d['birthplace'] = "NA"
          # d['principal_residence'] = "NA"
          # d['current_residence'] = "NA"
          d['original_speaker_id'] = e_sentence[n]
          speaker_list2.append(d)    
    # print(p_name)    
  except:
    print("except")
    # 진술인 참고인 
    for f in range(len(s_sentence)):
      if s_sentence[f].startswith("◯") and (s_sentence[f].endswith("참고인") or s_sentence[f].endswith("진술인")):
        role = s_sentence[f].split(" ")[1]            
        ss_start = f + 1         
    if ss_start != False:
      # print(s_start)
      for j in range(len(s_sentence)):
        if j >= ss_start:
          ee = s_sentence[j].split("(")
          eee = ee[0].split('\n')
          e_sentence += eee      
      for n in range(len(e_sentence)) :                
        if e_sentence[n] not in p_name:        
          d = {}
          d['id'] = e_sentence[n]
          # d['age'] = "NA"
          d['occupation'] = role  
          # d['sex'] = "NA"
          # d['birthplace'] = "NA"
          # d['principal_residence'] = "NA"
          # d['current_residence'] = "NA"
          d['original_speaker_id'] = e_sentence[n]
          speaker_list2.append(d)    
    # print(p_name)
  finally:
    for last in range(len(speaker_list2)):
      if speaker_list2[last]['id'] not in p2_name:
        d = {}
        d['id'] = speaker_list2[last]['id']
        # d['age'] = "NA"
        d['occupation'] = speaker_list2[last]['occupation']
        # d['sex'] = "NA"
        # d['birthplace'] = "NA"
        # d['principal_residence'] = "NA"
        # d['current_residence'] = "NA"
        d['original_speaker_id'] = speaker_list2[last]['id']  
        speaker_list2__2.append(d)     
  return speaker_list2__2
speaker_list2__2 = speaker_extractor2_or(text,speaker_list, speaker_list__, speaker_ex)
# pprint(speaker_list2__2)

[{'id': '차인순', 'occupation': '입법심의관', 'original_speaker_id': '차인순'}]


In [55]:
# json 형태로 변환하기 위한 틀
def convert_to_korea(speaker_list__, speaker_list2__2, dialog_json, id):
    # k = 0
    id = "SBRW 2100000001"
    # id_code = korea_formatted_data['id']
    # id = id_code.split(" ")
    # id0 = id[0]
    # id1 = int(id[1]) + k
    # id = id0 + str(id1)
    # korea_formatted_data['id'] = id    
    # k += 1

    metadata = {}

    doc = {}
    doc['id'] = id+".1"
    doc_metadata = {}

    doc_metadata['speaker'] = speaker_list__ + speaker_list2__2

    pprint(doc_metadata['speaker'])
    # doc_metadata['setting'] = {}
    # doc_metadata['setting']['relation'] = "NA"
    
    # doc_metadata['file_id'] = meta['파일명']
      # '회의록 원문 자료 파일명:'

    doc['metadata'] = doc_metadata
    
    # doc['paragraph'] = []
    # paragraph = {}
    # paragraph['id'] = ""
    # paragraph['form'] = ""
    # paragraph['original_form'] = ""
    # doc['paragraph'].append(paragraph)
    
    utterance = []
    
    n = 0
    for i in dialog_json:
        utter = {}
        utter['id'] = id+".1.1."+ str(n+1)
        utter['form'] = i['utterance'].rstrip()
        utter['original_form'] = i['utterance_or'].rstrip()
        utter['speaker_id'] = i['speaker']
        # utter['note'] = ""
        utterance.append(utter)
        n += 1
        # pprint(utter['id'])   
    doc['utterance'] = utterance
    
    d = {}
    d['id'] = id
    d['metadata'] = metadata
    d['document'] = doc
    
    return d
korea_formatted_data = convert_to_korea(speaker_list__, speaker_list2__2, dialog_json, id)
# pprint(korea_formatted_data)
# pprint(korea_formatted_data['document']['utterance'][0])

[{'id': '김상희', 'occupation': '소위원장', 'original_speaker_id': '김상희'},
 {'id': '김귀순', 'occupation': '수석전문위원', 'original_speaker_id': '김귀순'},
 {'id': '김옥이', 'occupation': '위원', 'original_speaker_id': '金玉伊'},
 {'id': '김태석', 'occupation': '여성부기획조정실장', 'original_speaker_id': '김태석'},
 {'id': '이정선', 'occupation': '위원', 'original_speaker_id': '이정선'},
 {'id': '곽정숙', 'occupation': '위원', 'original_speaker_id': '곽정숙'},
 {'id': '손숙미', 'occupation': '위원', 'original_speaker_id': '손숙미'},
 {'id': '이복실', 'occupation': '여성부권익증진국장', 'original_speaker_id': '이복실'},
 {'id': '정봉협', 'occupation': '여성부여성정책국장', 'original_speaker_id': '정봉협'},
 {'id': '김춘진', 'occupation': '위원', 'original_speaker_id': '김춘진'},
 {'id': '이성선', 'occupation': '여성부인력개발사업과장', 'original_speaker_id': '이성선'},
 {'id': '김권영', 'occupation': '여성부복지지원과장', 'original_speaker_id': '김권영'},
 {'id': '차인순', 'occupation': '입법심의관', 'original_speaker_id': '차인순'}]


In [57]:
def convert_korea_speaker(speaker_list__,speaker_list2__2):
  d = {}
  speaker = speaker_list__ + speaker_list2__2      
  d['speaker'] = speaker
  # d = speaker
  # pprint(speaker_list2__2)
  return d
korea_speaker = convert_korea_speaker(speaker_list__,speaker_list2__2)
# pprint(korea_speaker) 

In [61]:
# json 형태로 변환
def speaker_id_json(id):
  dd = []
  k = 0
  files = sorted(glob('/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 1차/*.txt'))  
  for fname in files:
    with open(fname, 'r', encoding='utf-8-sig') as file:
      txt = file.readlines()      
      pass
      text = clean_up(txt)      
      text_or = clean_up_or(txt)      
      dialog = dialog_extractor(text)      
      dialog_or = dialog_extractor_or(text_or)      
      speaker_ex = speaker_extractor(dialog)
      speaker_ex_or = speaker_extractor_or(dialog_or)      
      utterance_ex = utterance_extractor(dialog)       
      utterance_ex_or = utterance_extractor_or(dialog_or)         
      dialog_json = dialog_formatting(utterance_ex, utterance_ex_or)
      speaker_list__ = speaker_list_extractor_or(speaker_ex,speaker_ex_or)      
      speaker_list2__2 = speaker_extractor2_or(text,speaker_list, speaker_list__, speaker_ex)
      korea_speaker = convert_korea_speaker(speaker_list__,speaker_list2__2)
      d = {}
      id = "SBRW 2100000001"
      id_code = id.split(" ")
      id0 = id_code[0]
      id1 = int(id_code[1]) + k
      id_code = id0 + str(id1)        
      k += 1       
      d[id_code] = korea_speaker   
      dd.append(d)
  with open('/content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/회의록 1차 json/speakerlist.json', 'w') as f:
      json.dump(dd, f, ensure_ascii=False, indent=4)
    
    # /content/drive/My Drive/Colab Notebooks/task/Minutes(Korean)/(jw)pm_summary_preprocessor/
speaker_id_json(id)